In [8]:
import numpy as np
import pandas as pd
import os
import cv2

In [9]:
classified_image=["train","test"]
categories=["angry","disgust","fear","happy","neutral","sad","surprise"]
category_index={"angry":0,"disgust":1,"fear":2,"happy":3,"neutral":4,"sad":5,"surprise":6}

# Loading model

In [10]:
from keras.models import model_from_json
# load json and create model
file = open('json_file_path', 'r')
model_json = file.read()
file.close()
loaded_model = model_from_json(model_json)
# load weights
loaded_model.load_weights('h5_file')

In [11]:
loaded_model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_44 (Conv2D)           (None, 46, 46, 64)        640       
_________________________________________________________________
batch_normalization_28 (Batc (None, 46, 46, 64)        256       
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 44, 44, 64)        36928     
_________________________________________________________________
batch_normalization_29 (Batc (None, 44, 44, 64)        256       
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 22, 22, 64)        0         
_________________________________________________________________
dropout_26 (Dropout)         (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 20, 20, 128)      

# Testing


In [12]:
faceCascade = cv2.CascadeClassifier( os.path.join(cv2.data.haarcascades, "haarcascade_frontalface_default.xml") )

In [13]:
cap=cv2.VideoCapture(0)
cap.set(3,640)
cap.set(4,480)
cap.set(10,180)

False

In [14]:
if not cap.isOpened():
    print("Cannot open camera")
    exit()

    
font=cv2.FONT_HERSHEY_COMPLEX
categories=np.array(categories)

while True:
    success,result=cap.read()
    
    if not success:
        print("Can't recieve the signals....")
        break
        
    
    
    grayimg=cv2.cvtColor(result,cv2.COLOR_BGR2GRAY)
    
    faces=faceCascade.detectMultiScale(
    grayimg,
    scaleFactor=1.1,
    minNeighbors=4,
    minSize=(30,30),
    flags=cv2.CASCADE_SCALE_IMAGE
    )
    
    img=result.copy()
    img=cv2.resize(img,(48,48))
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    
    cv2.imshow("Processed Image",img)
    
    img=np.reshape(img,(1,48,48,1))
    
    #making our model to predict
    predictions=loaded_model.predict(img)
    prob=np.argmax(predictions[0])
    category_prediction=categories[prob]
    
    
    #this code is to make rectangles around the face
    for (x,y,w,h) in faces:
        if(category_prediction=="angry"):
            color=(0,0,255)  #red color
        elif(category_prediction=="disgust"):
            color=(0,255,0)  #green color
        elif(category_prediction=="fear"):
            color=(0,0,0)    #black color
        elif(category_prediction=="happy"):
            color=(0,255,255) #yellow color
        elif(category_prediction=="neutral"):
            color=(153,255,255) #cream color
        elif(category_prediction=="sad"):
            color=(153,76,0)  #blue
        elif(category_prediction=="surprise"):
            color=(0,128,255) #orange
        
        cv2.rectangle(result,(x,y),(x+w,y+h),color,3)
        cv2.rectangle(result,(x,y-40),(x+w,y),color,-1)
        cv2.putText(result,str(((predictions[0][prob]*100)))+"%",(x+10,y-10),font,0.75,(255,255,255),2)
        cv2.rectangle(result,(x,y+h+3),(x+w,y+h+40),color,-1)
        cv2.putText(result,category_prediction,(x+10,y+h+45),font,0.75,(255,255,255),2)
        
        cv2.imshow("Result",result)
    if(cv2.waitKey(1) & 0xff==ord('q')):
        break
print("Press the 'q' button in your keyboard to exit")
cap.release()
cv2.destroyAllWindows()


Press the 'q' button in your keyboard to exit
